# **FNN**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 2 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_2s_all.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[3.6285463e-05, 3.8200385e-05, 4.0739291e-05, ..., 3.0273466e-06,
         2.2592510e-06, 2.1416045e-06],
        [7.4347768e-09, 8.2490290e-09, 9.6704795e-09, ..., 2.0450429e-07,
         2.5535245e-08, 2.3674612e-08],
        [1.6406947e-06, 1.6677786e-06, 1.7138188e-06, ..., 2.2908193e-06,
         1.6840297e-06, 1.5936671e-06],
        ...,
        [2.7316248e-06, 2.9339224e-06, 3.2875062e-06, ..., 1.0139429e-06,
         5.8416873e-07, 5.6021997e-07],
        [2.4674987e-06, 2.5058257e-06, 2.5689174e-06, ..., 9.9417584e-06,
         2.3140782e-07, 1.8963851e-07],
        [3.0352521e-06, 4.1363774e-06, 6.0993666e-06, ..., 2.6265046e-05,
         7.4409758e-07, 7.3155365e-07]], dtype=float32),
 'mfcc': array([[-1.0000000e+00, -2.3037937e-01, -5.0594997e-01, ...,
          2.8443993e-03,  1.4909296e-02,  8.9377072e-03],
        [-1.0000000e+00, -2.6351056e-01, -5.3618097e-01, ...,
          5.8985357e-03,  1.1296259e-02,  1.5974644e-03],
        [-1.0000000e

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(22631,)

array([16, 18, 16,  8,  0,  7,  1, 13,  5,  0,  0, 10,  0,  3,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(22631, 60)

array([3.4329585e-06, 3.5621692e-06, 3.7917514e-06, 4.1497665e-06,
       4.7353369e-06, 6.1132328e-06, 1.1151645e-05, 3.9353021e-05,
       3.6168797e-04, 1.5898738e-02, 8.7700933e-02, 3.1693410e-02,
       6.7264899e-03, 1.9934600e-02, 6.2744088e-02, 1.4736451e-01,
       2.4715923e-01, 4.3244416e-01, 6.4362824e-01, 7.3113507e-01,
       6.8003190e-01, 5.6165701e-01, 5.0011736e-01, 4.7874513e-01,
       4.9124286e-01, 4.3701285e-01, 3.3579260e-01, 2.3378754e-01,
       1.5629211e-01, 7.9104789e-02, 4.6607573e-02, 2.8233325e-02,
       2.3269402e-02, 2.1181842e-02, 1.6698154e-02, 1.0057325e-02,
       7.4147303e-03, 5.1149242e-03, 3.2818515e-03, 2.6066876e-03,
       2.0486056e-03, 1.5702612e-03, 1.3500779e-03, 1.2921348e-03,
       1.4930066e-03, 8.4594055e-04, 7.1877841e-04, 6.1083917e-04,
       5.5965967e-04, 6.8108435e-04, 5.1947287e-04, 4.0253717e-04,
       2.7309530e-04, 1.7824488e-04, 7.9370431e-05, 1.4202556e-05,
       8.8283167e-07, 4.1159396e-08, 2.1225540e-08, 2.0640552e

'mfcc'

(22631, 20)

...

(22631, 12)

array([0.60293585, 0.7147668 , 0.7193865 , 0.67120546, 0.58156115,
       0.5468347 , 0.5731501 , 0.54188204, 0.4656724 , 0.40363166,
       0.384592  , 0.4238847 ], dtype=float32)

'rms'

(22631, 1)

array([0.02097528], dtype=float32)

In [13]:
display(val_y.shape)
display(val_y[:15])

(6044,)

array([10,  2, 12,  0,  1, 18,  2,  2, 17,  2,  2,  2,  0, 17,  7],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(6044, 60)

array([8.3357208e-06, 8.4720250e-06, 8.7100416e-06, 9.0726417e-06,
       9.5879695e-06, 1.0466226e-05, 1.3052877e-05, 2.5488100e-05,
       7.8058765e-05, 2.7748491e-04, 1.2698962e-03, 8.1089614e-03,
       2.6884841e-02, 7.7197373e-02, 1.1425581e-01, 1.5141682e-01,
       2.0923702e-01, 2.1435727e-01, 2.4789700e-01, 2.3410201e-01,
       2.7105078e-01, 2.9080284e-01, 3.2453740e-01, 2.5168687e-01,
       2.6569101e-01, 3.2397676e-01, 3.5192734e-01, 3.7910870e-01,
       3.8736138e-01, 3.8108847e-01, 3.9733896e-01, 4.0133148e-01,
       3.4785292e-01, 2.9322231e-01, 2.5932199e-01, 2.6791215e-01,
       3.5579714e-01, 3.4876502e-01, 1.5109749e-01, 1.0652289e-01,
       7.9575814e-02, 6.9220871e-02, 6.0770944e-02, 5.9468258e-02,
       5.7272933e-02, 5.1554888e-02, 5.1538818e-02, 5.3031396e-02,
       5.8848843e-02, 5.2466441e-02, 5.0471891e-02, 6.2297914e-02,
       6.0394101e-02, 4.1807249e-02, 1.9994956e-02, 4.4141770e-03,
       2.8683330e-04, 6.7595729e-06, 1.3183726e-06, 1.1420557e

'mfcc'

(6044, 20)

...

(6044, 12)

array([0.50444525, 0.50069445, 0.4891589 , 0.45970854, 0.46573985,
       0.48136368, 0.57291734, 0.5993228 , 0.4840322 , 0.49154848,
       0.5054053 , 0.4910522 ], dtype=float32)

'rms'

(6044, 1)

array([0.0361241], dtype=float32)

## **FNN Model**

In [15]:
train_results = {}
val_results = {}
k = 15

### **With all the features**

In [16]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)

training_features.shape

(22631, 93)

In [17]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)

validation_features.shape

(6044, 93)

Fit the model with training data

In [18]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [19]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3446393117140966


In [20]:
train_yhat_result = model.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3120
           1       1.00      1.00      1.00      2276
           2       1.00      1.00      1.00      1559
           3       1.00      1.00      1.00      1459
           4       1.00      1.00      1.00       223
           5       1.00      1.00      1.00       362
           6       1.00      1.00      1.00       434
           7       1.00      1.00      1.00      1295
           8       1.00      1.00      1.00       381
           9       1.00      1.00      1.00       658
          10       1.00      1.00      1.00       904
          11       1.00      1.00      1.00       349
          12       1.00      1.00      1.00      2234
          13       1.00      1.00      1.00       621
          14       1.00      1.00      1.00       891
          15       1.00      1.00      1.00      1310
          16       1.00      1.00      1.00     

In [21]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.45      0.36       834
           1       0.16      0.38      0.23       384
           2       0.66      0.23      0.34      1441
           3       0.59      0.79      0.67       307
           4       0.20      0.05      0.08       154
           5       0.43      0.56      0.49        64
           6       0.04      0.03      0.04        65
           7       0.58      0.57      0.58       354
           8       0.18      0.35      0.24       127
           9       0.86      0.65      0.74       210
          10       0.21      0.14      0.17       243
          11       0.10      0.07      0.08       122
          12       0.18      0.18      0.18       287
          13       0.14      0.05      0.08       114
          14       0.02      0.05      0.02        65
          15       0.54      0.45      0.49       438
          16       0.55      0.18      0.28   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(22631, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(6044, 20)

In [24]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [25]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.27183984116479154


## **With MFCCs and Mel-Spectrogram**

In [26]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(22631, 80)

In [27]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(6044, 80)

In [28]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [29]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3330575777630708


## **With MFCCs and Chroma**

In [30]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(22631, 32)

In [31]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(6044, 32)

In [32]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [33]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31353408338848443


## **With MFCCs and RMS**

In [34]:
training_features = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)

training_features.shape

(22631, 21)

In [35]:
validation_features = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)

validation_features.shape

(6044, 21)

In [36]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [37]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_rms'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_rms'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2814361350099272


## **With Mel-Spectrogram, MFCCs, RMS**

In [38]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)

training_features.shape

(22631, 81)

In [39]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)

validation_features.shape

(6044, 81)

In [40]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [41]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3441429516876241


## **With Mel-Spectrogram, MFCCs, Chroma**

In [42]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(22631, 92)

In [43]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(6044, 92)

In [44]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(n_neighbors=k, weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(n_neighbors=15, weights='distance')

In [45]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.33504301786896096


# Review results from all models

In [46]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.34
6,melspectrogram_chroma_mfcc,1.0,0.34
5,melspectrogram_rms_mfcc,1.0,0.34
1,mfcc,1.0,0.27
3,mfcc_chroma,1.0,0.31
2,mfcc_melspectrogram,1.0,0.33
4,mfcc_rms,1.0,0.28
